# Woodstock

## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
        #!apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("ipopt") or os.path.isfile("ipopt"))

from pyomo.environ import *

     |████████████████████████████████| 9.0MB 2.5MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 256kB 45.5MB/s 
     |████████████████████████████████| 163kB 46.8MB/s 


# Woodstock


The Woodstock Appliance Company wants your help in planning their annual inventory ordering scheme. A basic inventory problem requires balancing ordering costs and carrying costs to minimize total cost.  We are given a product's annual demand $D$ and we need to determine the quantity to order $Q$.

Suppose that a fixed ordering cost of $K$ is incurred with each order, independent of the order size.  If the quantity ordered is $Q$ then the number of orders placed per year in $\frac{D}{Q}$.  It then follows that the ordering cost per year is

$
\textrm{Ordering cost} = \left( \textrm{Cost per order} \right) \cdot \left( \textrm{Orders per year} \right)  = \frac{KD}{Q}.
$

By placing orders so that replenishing occurs just as stock is depleted, we ensure that the inventory level fluctuates between a low of zero and a high of $Q$.  The average inventory is therefore $\frac{Q}{2}$.  Suppose also that items held in inventory incur an annual carrying cost of $h$.  (This cost is often expressed as a percentage of the product's unit cost, in the form $h = ic$, where $i$ denotes the carrying cost percentage, and $c$ denotes an item's cost.)  Then, the annual inventory carrying costs are

$
\textrm{Inventory cost} = \left( \textrm{Carrying charge} \right) \cdot \left( \textrm{Average inventory} \right)  = \frac{hQ}{2}.
$

Then, the average total cost (ATC) is the sum of the two components:

$
\textrm{ATC} = \left( \textrm{Ordering cost} \right) + \left( \textrm{Inventory cost} \right)  = \frac{KD}{Q} + \frac{hQ}{2}.
$

This is for each product, but Woodstock needs to consider all four of the products that it sells.  The annual demands for these products range from 300 per year for a high-end vacuum cleaner to 30,000 per year for a table fan.  The order cost, holding cost, and purchase cost are known for each of the four products as well as how much space each product occupies.  The numerical information is summarized in the following table.

Product	|	1	|	2	|	3	|		4
---| --- | --- | --- | ---
Demand per year	|	5,000	|	10,000	|	30,000	| 300
Cost per order	|	\$400	|	\$700	|	\$100	| \$250
Carrying cost percentage	|	10\%	|	10\%	|	10\%	| 10\%
Item purchase cost	|	\$500	|	\$250	|	\$80	| \$1,000
Space per unit	|	12ft$^2$	|	25ft$^2$	|	5ft$^2$	| 20ft$^2$


Woodstock stores inventory in its warehouse, which contains 12,000 square feet that can be dedicated to any combination of the four products.  The problem is to find the order quantities that minimize cost which respecting the limit on storage space. What is the optimal ordering order quantities to minimize ATC$_1$ + ATC$_2$ + ATC$_3$ + ATC$_4$, where ATC$_i$ is the sum of the ordering cost and the inventory cost for product $i$?


**Objective Function**

$\min atc_1 + atc_2 + atc_3 + atc_4$ `(objective function)`

**Write the Constraints**

subject to:

* $ord_i = \frac{K_i \cdot D_i}{q_i}, i \in [4]$ `(ordering cost)`
* $inv_i = \frac{h_i \cdot q_i}{2}, i \in [4]$  `(inventory cost)`
* $atc_i = ord_i + inv_i, i \in [4]$ `(total cost)`
* $12 \cdot q_1 + 25 \cdot q_2 +  5 \cdot q_3 + 20 \cdot q_4 \leq 12,000$ `(warehouse capacity)`

`Domains`
* $q_i \in \mathbb{R}^+, i \in [4]$ `(quantity ordered)`
* $ord_i \in \mathbb{R}^+, i \in [4]$ `(ordering cost)`
* $inv_i \in \mathbb{R}^+, i \in [4]$ `(inventory cost)`
* $atc_i \in \mathbb{R}^+, i \in [4]$ `(order volume variables)`


In [ ]:
Products = [0,1,2,3]
D = [5000,	10000,	30000,	300]
K = [400,	700,	100,	250]
i = [0.1,	0.1,	0.1,	0.1]
c = [500,	250,	80,	1000]
S = [12,	25,	5,	20]
T = 12000

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables  - Initialize variable to 1.0
model.q = Var(Products,domain=NonNegativeReals,initialize=1.0)
model.ord = Var(Products,domain=NonNegativeReals,initialize=1.0)
model.inv = Var(Products,domain=NonNegativeReals,initialize=1.0)
model.atc = Var(Products,domain=NonNegativeReals,initialize=1.0)


# Constraints
model.constraints = ConstraintList()

space_expr = 0
for product in Products:
  model.constraints.add(model.ord[product] == (K[product]*D[product])/model.q[product])
  model.constraints.add(model.inv[product] == (i[product]*c[product]*model.q[product])/2)
  model.constraints.add(model.atc[product] == model.inv[product] + model.ord[product])
  space_expr += S[product]*model.q[product]

model.constraints.add(space_expr <= T)


# declare objective
obj_expr = 0
for product in Products:
  obj_expr += model.atc[product]
model.cost = Objective(
                      expr = obj_expr,
                      sense = minimize)

# show the model you've created
model.pprint()

5 Set Declarations
    atc_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
    inv_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}
    ord_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}
    q_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

4 Var Declarations
    atc : Size=4, Index=atc_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :  None : False : False : NonNegativ

In [ ]:
# solve it
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
# show the results
print("Objective value = ", model.cost())
print("Product 1 = ", model.q[0]())
print("Product 2 = ", model.q[1]())
print("Product 3 = ", model.q[2]())
print("Product 4 = ", model.q[3]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 16
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02496647834777832
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objec